### Combine Crossref Awards and Backfill Awards to create single Awards table

In [0]:
%sql
CREATE OR REPLACE TABLE openalex.awards.openalex_awards
USING delta
AS
WITH combined AS (
  SELECT
    abs(xxhash64(id)) % 9000000000 as id,
    NULL as display_name,
    NULL as description,
    funder_id,
    funder_award_id,
    NULL as amount,
    NULL as currency,
    struct(
      funder.id,
      funder.display_name,
      funder.ror_id,
      funder.doi
    ) as funder,
    NULL as funding_type,
    NULL as funder_scheme,
    'cited' as provenance,
    NULL as start_date,
    NULL as end_date,
    NULL as start_year,
    NULL as end_year,
    NULL as lead_investigator,
    NULL as co_lead_investigator,
    NULL as investigators,
    NULL as landing_page_url,
    NULL as doi,
    created_date,
    updated_date,
    2 as priority  -- lower priority for backfill
  FROM openalex.awards.backfill_awards
  
  UNION ALL
  
  SELECT 
    abs(xxhash64(id)) % 9000000000 as id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    created_date,
    updated_date,
    1 as priority  -- higher priority for crossref
  FROM openalex.awards.crossref_awards
)
SELECT 
  * EXCEPT(priority, row_num)
FROM (
  SELECT 
    *,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY priority) as row_num
  FROM combined
)
WHERE row_num = 1